### 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [7]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [8]:
data = pd.merge(ratings, movies)

In [9]:
data

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


### 2) 분석

In [10]:
# 유저 수
data['user_id'].nunique()

6039

In [11]:
# 영화 개수
data['movie_id'].nunique()

3628

In [12]:
# 인기 많은 영화
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3) 내가 선호하는 영화 5가지 골라서 ratings에 추가

In [13]:
my_favorite = ['Jurassic World (2015)' , 'Finding Nemo (2003)' ,'Finding Dory (2016)' ,'Up (2009)', 'A Simple Favor (2018)']

# 'simdrone'이라는 user_id가 위 영화들을 30회씩 봤다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['simdrone']*5, 'title': my_favorite, 'counts':[30]*5})

if not data.isin({'user_id':['simdrone']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,simdrone,NaN,30,NaN,Jurassic World (2015),NaN
1,simdrone,NaN,30,NaN,Finding Nemo (2003),NaN
2,simdrone,NaN,30,NaN,Finding Dory (2016),NaN
3,simdrone,NaN,30,NaN,Up (2009),NaN
4,simdrone,NaN,30,NaN,A Simple Favor (2018),NaN


In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['simdrone'])
print(title_to_idx['Jurassic World (2015)'])

6039
3628


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,1,1193.0,5,978298413.0,0,Drama
2,2,1193.0,4,978220179.0,0,Drama
3,3,1193.0,4,978199279.0,0,Drama
4,4,1193.0,5,978158471.0,0,Drama
...,...,...,...,...,...,...
0,6039,NaN,30,NaN,3628,NaN
1,6039,NaN,30,NaN,3629,NaN
2,6039,NaN,30,NaN,3630,NaN
3,6039,NaN,30,NaN,3631,NaN


In [17]:
using_cols = ['user_id', 'title', 'counts']
data = data[using_cols]
data

,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,3628,30
1,6039,3629,30
2,6039,3630,30
3,6039,3631,30


### 4) CSR matrix 직접 만들기

In [18]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlthernatingLeastSquares 모델 직접 구성하여 훈련

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나 골라 훈련된 모델이 예측한 나의 선호도 파악

In [27]:
simdrone, Jurassic_World_2015 = user_to_idx['simdrone'], title_to_idx['Jurassic World (2015)']
simdrone_vector, Jurassic_World_2015_vector = als_model.user_factors[simdrone], als_model.item_factors[Jurassic_World_2015]

In [28]:
simdrone_vector

array([-4.4376788e-01, -6.3284880e-01,  2.0500951e+00, -8.9401615e-01,
       -2.0110202e+00, -7.7843331e-02,  2.1470132e+00, -1.5221422e+00,
        1.0544873e-01,  2.3949428e+00, -1.5019339e+00,  3.7203228e-01,
       -1.2876285e+00,  1.9650918e+00, -1.1115288e+00,  3.2013634e-01,
       -4.1652817e-01,  3.2362795e-01,  7.5106704e-01,  5.0607204e-01,
       -1.6358145e+00,  1.5429226e+00,  9.7596568e-01, -4.5425972e-01,
       -6.3894653e-01, -7.2301567e-01,  1.4019852e+00,  3.2329760e+00,
       -1.8267351e+00,  1.5498644e-01, -4.7065780e-01, -2.0614018e+00,
        2.7498433e-01, -4.3856966e-01, -1.0668881e+00, -1.5139822e+00,
       -1.1506369e+00, -2.7900224e+00,  2.8586802e-01, -2.0963611e-01,
       -8.9028347e-01,  6.8586910e-01, -1.9765560e-01, -1.6081227e+00,
        1.6654281e-02, -7.3208189e-01,  1.2190697e+00, -2.4158072e+00,
       -2.8844222e-01,  8.3990872e-01,  8.0514663e-01, -2.5013152e-01,
        4.4224563e-01,  2.0809233e+00,  2.7452856e-01,  1.9048969e+00,
      

In [29]:
Jurassic_World_2015_vector

array([ 2.3554789e-03,  6.4808177e-04,  6.1561833e-03,  7.6651073e-04,
       -3.6026950e-03,  3.1173246e-03,  7.3542343e-03, -2.6460041e-03,
        3.2685748e-03,  7.2041154e-03, -2.8101317e-04,  2.9084999e-03,
       -3.3380315e-03,  6.1176862e-03, -2.1180464e-04,  8.8758685e-04,
        1.9063514e-03,  3.7814011e-03,  3.5349696e-03,  3.5654192e-03,
       -6.1992032e-04,  6.9416016e-03,  4.7411490e-03, -4.2470705e-04,
        9.7770081e-04, -5.9705988e-05,  5.0197393e-03,  1.0320255e-02,
       -7.6388073e-04,  3.0229317e-03,  1.0923186e-03, -4.2182226e-03,
        1.6243006e-03,  1.6247591e-03,  1.0722397e-03, -6.7291065e-04,
       -1.2701699e-03, -4.8571355e-03,  4.1488535e-03,  9.0045843e-04,
        6.9344358e-04,  2.6902505e-03,  2.0141345e-03, -2.6453540e-03,
        1.2222595e-03,  1.3581083e-04,  5.5570300e-03, -3.4464023e-03,
       -3.6033179e-04,  3.3059109e-03,  4.5812326e-03,  1.4895934e-03,
        1.1438748e-03,  6.3766507e-03,  2.6466150e-03,  6.1458172e-03,
      

In [30]:
np.dot(simdrone_vector, Jurassic_World_2015_vector)

0.37445742

In [32]:
Jurassic_Park_1993 = title_to_idx['Jurassic Park (1993)']
Jurassic_Park_1993_vector = als_model.item_factors[Jurassic_Park_1993]
np.dot(simdrone_vector, Jurassic_Park_1993_vector)

0.018117303

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아보기

In [34]:
favorite_title = 'Jurassic Park (1993)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(107, 1.0),
 (175, 0.80194575),
 (92, 0.71538424),
 (62, 0.5653566),
 (124, 0.51311916),
 (87, 0.4607705),
 (82, 0.43448836),
 (3466, 0.42805508),
 (150, 0.3948916),
 (145, 0.37335396),
 (542, 0.3708807),
 (58, 0.36262828),
 (1754, 0.36163273),
 (80, 0.3408296),
 (206, 0.33417594)]

In [35]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Braveheart (1995)',
 'Lost World: Jurassic Park, The (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Independence Day (ID4) (1996)',
 'Fifth Element, The (1997)',
 'Rocky Horror Picture Show, The (1975)',
 'Mission: Impossible (1996)',
 'Little Odessa (1994)',
 'Stand by Me (1986)',
 'Love! Valour! Compassion! (1997)']

In [36]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [37]:
get_similar_title('E.T. the Extra-Terrestrial (1982)')

['E.T. the Extra-Terrestrial (1982)',
 'Close Encounters of the Third Kind (1977)',
 'Back to the Future (1985)',
 'Dark Crystal, The (1982)',
 'Big (1988)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Time Bandits (1981)',
 'Cocoon (1985)',
 'Dead Poets Society (1989)']

In [38]:
get_similar_title('Ghostbusters (1984)')

['Ghostbusters (1984)',
 "Ferris Bueller's Day Off (1986)",
 'Back to the Future (1985)',
 'Big (1988)',
 'Who Framed Roger Rabbit? (1988)',
 'Beetlejuice (1988)',
 'Fish Called Wanda, A (1988)',
 'Shining, The (1980)',
 'Night of the Living Dead (1968)',
 'Bull Durham (1988)']

### 8) 내가 가장 좋아할 만한 영화들 추천받기

In [39]:
user = user_to_idx['simdrone']
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(669, 0.56495214),
 (453, 0.5191747),
 (391, 0.48436272),
 (474, 0.4352759),
 (253, 0.39583394),
 (87, 0.39036345),
 (984, 0.38561755),
 (790, 0.38506132),
 (550, 0.38306794),
 (421, 0.38210875),
 (250, 0.37516022),
 (91, 0.36141056),
 (691, 0.35773653),
 (224, 0.3422922),
 (615, 0.33607495),
 (890, 0.33480185),
 (4, 0.32279566),
 (768, 0.32053587),
 (381, 0.30887842),
 (332, 0.303476)]

In [40]:
[idx_to_title[i[0]] for i in title_recommended]

['Man on the Moon (1999)',
 'Boogie Nights (1997)',
 'Cider House Rules, The (1999)',
 'Chicken Run (2000)',
 'Mystery Men (1999)',
 'Braveheart (1995)',
 'Dangerous Liaisons (1988)',
 'Birdcage, The (1996)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Midnight in the Garden of Good and Evil (1997)',
 'Edward Scissorhands (1990)',
 'Close Encounters of the Third Kind (1977)',
 'Dune (1984)',
 'L.A. Confidential (1997)',
 'Cocoon (1985)',
 'Professional, The (a.k.a. Leon: The Professional) (1994)',
 "Bug's Life, A (1998)",
 'Get Shorty (1995)',
 'Elizabeth (1998)',
 'Moonstruck (1987)']

In [42]:
Sixth_Sense_1999 = title_to_idx['Sixth Sense, The (1999)']
explain = als_model.explain(user, csr_data, itemid=Sixth_Sense_1999)

In [43]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Up (2009)', -0.008241319064465092),
 ('Finding Nemo (2003)', -0.008249904769234158),
 ('Finding Dory (2016)', -0.008468803082731078),
 ('A Simple Favor (2018)', -0.009092090747675974),
 ('Jurassic World (2015)', -0.009482939923616717)]